In [26]:
import os

import pandas as pd
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nfl_data_py as nfl

np.float_ = np.float64


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.mode.copy_on_write = True

In [2]:
years = range(2009, 2024)

In [30]:

for year in years:
    if not os.path.exists(f'../.cache/pbp/season={year}'):
        os.makedirs(f'../.cache/pbp/season={year}')
        nfl.cache_pbp(years=[year], alt_path='../.cache/pbp', downcast=True)

df_pbp = nfl.import_pbp_data(years, cache=True, alt_path='../.cache/pbp')

df_schedules = nfl.import_schedules(years).reset_index(drop=True)

df_stadiums_locs = pd.read_csv('../data/stadiums.csv')[['stadium_id','lat','lon','timezone']]

df_elevations = pd.read_csv('../data/stadium_elevation.csv')

cols = ['season','player_id','player_name','years_exp','draft_number','age']
df_rosters = nfl.import_seasonal_rosters(years)[cols]

2009 done.
2010 done.
2011 done.
2012 done.
2013 done.
2014 done.
2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
Downcasting floats.


In [ ]:
df_pbp.drive.value_counts()

sp
0.0    665339
1.0     53252
Name: count, dtype: int64

In [31]:
df_pbp.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,td_player_name,td_player_id,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,vegas_wpa,vegas_home_wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,passer_player_id,passer_player_name,passing_yards,receiver_player_id,receiver_player_name,receiving_yards,rusher_player_id,rusher_player_name,rushing_yards,lateral_receiver_player_id,lateral_receiver_player_name,lateral_receiving_yards,lateral_rusher_player_id,lateral_rusher_player_name,lateral_rushing_yards,lateral_sack_player_id,lateral_sack_player_name,interception_player_id,interception_player_name,lateral_interception_player_id,lateral_interception_player_name,punt_returner_player_id,punt_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,kickoff_returner_player_name,kickoff_returner_player_id,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tack

In [ ]:
# Columns to keep
variables = [
    'season', 'game_id', 'play_id', 'week', 'home_team', 'away_team', 'season_type', 'posteam',
    'posteam_type', 'defteam', 'game_half', 'game_seconds_remaining', 'score_differential',
    'yardline_100', 
] # score_differential = posteam_score - defteam_score

# Sort by game_id and play_id and create columns for previous play's timeout and description
df_pbp = df_pbp.sort_values(['game_id', 'play_id'])
df_pbp['timeout_prev_play'] = df_pbp['timeout'].shift(1)
df_pbp['timeout_team_prev_play'] = df_pbp['timeout_team'].shift(1)
df_pbp['desc_prev_play'] = df_pbp['desc'].shift(1)

# Filter for field goal attempts
df_fg = df_pbp[df_pbp['field_goal_attempt'] == True][variables].reset_index(drop=True)

# Fix missing timeout team data for specific plays
fixes = {
    ('2013_07_SD_JAX', 2389.0): 'JAX',
    ('2015_09_JAX_NYJ', 2731.0): 'JAX'
}
for (game_id, play_id), team in fixes.items():
    df_fg.loc[(df_fg.play_id == play_id) & (df_fg.game_id == game_id), 'timeout_team_prev_play'] = team

# Determine if kicker was iced
df_fg['iced'] = np.where(
    (df_fg['timeout_prev_play'] == True) & (df_fg['posteam'] != df_fg['timeout_team_prev_play']), 
    1, 
    0
)

# Create binary columns for various conditions
df_fg['overtime'] = (df_fg['game_half'] == 'Overtime').astype(int)
df_fg['postseason'] = (df_fg['season_type'] == 'POST').astype(int)
df_fg['home_stadium'] = (df_fg['posteam_type'] == 'home').astype(int)
df_fg['grass_surface'] = df_fg['surface'].isin(['grass', 'grass ', 'dessograss']).astype(int)
df_fg['outdoors'] = (df_fg['roof'].isin(['outdoors', 'open'])).astype(int)

# Handle missing temperature and wind data
df_fg['temp'] = df_fg['temp'].fillna(70)
df_fg['wind'] = df_fg['wind'].fillna(0)

# Fill missing surface data based on the most common surface for each stadium
stadiums_with_surface = df_fg.query('surface == ""').stadium_id.unique()
for stadium_id in stadiums_with_surface:
    v_counts = df_fg.query('stadium_id == @stadium_id').surface.value_counts()
    surface = v_counts.index[0] 
    if (surface == "") & (len(v_counts) > 1):
        surface = v_counts.index[1]
    df_fg.loc[(df_fg.stadium_id == stadium_id) & (df_fg.surface == ""), 'surface'] = surface

# Special case for a specific stadium
df_fg.loc[df_fg.stadium_id == "GER00", 'surface'] = "dessograss"

# Merge stadium elevations
df_fg = df_fg.merge(df_elevations, how='left', on='stadium_id')

# Create a binary column for field goal made
df_fg['field_goal_made'] = df_fg['field_goal_result'].apply(lambda x: 1 if x == 'made' else 0)

df_fg = df_fg.merge(df_schedules[['game_id', 'gameday', 'gametime']], on='game_id', how='left')

df_fg = df_fg.merge(df_stadiums_locs, how='left', on='stadium_id')

# Find the stadium elevation for the kicker's home stadium
home_stadiums = (df_schedules[['season', 'home_team', 'stadium_id']]
    .groupby(['season', 'home_team'])
    .agg(total=('stadium_id', 'count'), 
         home_stadium=('stadium_id', lambda x: x.value_counts().index[0]))
    .reset_index()
    .sort_values('total', ascending=True)
    [['season', 'home_team', 'home_stadium']]
    .merge(df_elevations, how='left', left_on='home_stadium', right_on='stadium_id')
    .drop(['stadium_id','home_stadium'], axis=1)
    .rename({'elevation':'home_stadium_elevation', 'home_team':'team'}, axis=1)
)
team_mapper = {
    'OAK': 'LV',
    'SD': 'LAC',
    'STL': 'LA'
}
home_stadiums['team'] = home_stadiums['team'].replace(team_mapper)
df_fg = df_fg.merge(home_stadiums, how='left', left_on=['season', 'posteam'], right_on=['season', 'team'])

df_fg['elevation_diff'] = df_fg['elevation'] - df_fg['home_stadium_elevation']

# Convert start time to datetime and convert to local time
def convert_timezone(row):
    return row['start_time'].tz_convert(pytz.timezone(row['timezone']))
df_fg['start_time'] = pd.to_datetime(df_fg.gameday + ' ' + df_fg.gametime, format='%Y-%m-%d %H:%M')
df_fg['start_time'] = df_fg['start_time'].dt.tz_localize(pytz.timezone('US/Eastern'))
df_fg['start_time'] = df_fg.apply(convert_timezone, axis=1)

# Join the roster data
df_fg = df_fg.merge(df_rosters, how='left', left_on=['season', 'kicker_player_id'], right_on=['season', 'player_id'])

pressure_situations = df_fg.overtime | \
    ((df_fg.score_differential <= 0) & (df_fg.score_differential >= -3) & (df_fg.game_seconds_remaining <= 500)) | \
    ((df_fg.postseason == 1) & (df_fg.score_differential <= 0) & (df_fg.score_differential >= -3)) | \
    (df_fg.iced == 1)

df_fg['pressure_situation'] = pressure_situations.astype(int)

# Drop unnecessary columns
drop_cols = [
    'kicker_player_name','game_half', 'season_type', 'posteam_type', 'surface', 'roof',
    'timeout_prev_play', 'timeout_team_prev_play', 'field_goal_result', 
    'field_goal_attempt', 'stadium_id', 'gameday', 'gametime', 'lat', 'lon', 'timezone', 'home_stadium_elevation',
    'start_time', 'player_id', 'team', 'posteam'
]

# Reorder columns
df_fg = df_fg[df_fg.columns.drop('player_name').insert(df_fg.columns.get_loc('kicker_player_id') - 1, 'player_name')]
df_fg = df_fg[df_fg.columns.drop('field_goal_made').insert(len(df_fg.columns)-1, 'field_goal_made')]
df_fg = df_fg[df_fg.columns.drop('years_exp').insert(df_fg.columns.get_loc('kicker_player_id') + 1, 'years_exp')]
df_fg = df_fg[df_fg.columns.drop('draft_number').insert(df_fg.columns.get_loc('kicker_player_id') + 2, 'draft_number')]
df_fg = df_fg[df_fg.columns.drop('age').insert(df_fg.columns.get_loc('kicker_player_id') + 3, 'age')]
df_fg = df_fg.drop(drop_cols, axis=1)